# The 20-Minute Modeling Tool

Let's build a modeling tool in 20 minutes!

We'll use [Neo4j](http://neo4j.com/) for data storage, [Python](https://www.python.org/) for data processing, [vis.js](http://visjs.org/) for data visualization.

## Neo4j

From [Wikipedia's Neo4j article](https://en.wikipedia.org/wiki/Neo4j) (with ephasis by me):

> Neo4j is an **open-source graph database** implemented in Java and **accessible from software written in other languages** using the **Cypher query language** through a transactional HTTP endpoint. The developers describe Neo4j as an ACID-compliant transactional database with native graph storage and processing. **Neo4j is the most popular graph database.**

A graph database is simply a set of **nodes** (things) and **relationships** (pairs of things). Both nodes and relationships can have labels and properties associated with them. Graph-traversal algorithms (both built-in and custom-built) read the properties of nodes and traverse the relationships between them to return nodes and relationships of interest.

## 20-second overview

In this session, we'll connect to the blank Neo4j database running on our local server.

Then, we'll populate it with nodes that will define:

* our metamodel \*
* our project model
* our project data

and relationships that will define:

* how our metamodel \* connects within itself
* how our project model conforms to our metamodel \* and connects within itself
* how our project data conforms to our project model and connects within itself

Finally, we'll query our database and obtain useful views of our data, as well as suggestions for what additional data could be defined (what to ask our stakeholders).

\* We'll use a sloppy subset of UML in this session, but any metamodel is possible - SysML, UML, multi-layered ontologies, etc.

## Jack and Jill and Kidney Failure

Jack needs a kidney transplant or he will die. We will find him a donor with modeling!

## Connect to the Neo4j Database

First we'll connect to the database and delete any existing nodes and relationships.

We'll communicate with Neo4j through the `py2neo` Python library.

In [1]:
from py2neo import authenticate, Graph
from py2neo.error import Unauthorized

graph = Graph()

try:
    # Clear any existing nodes and relationships in the database
    graph.delete_all()
except Unauthorized:
    # If you have a username/password, replace neo4j/password with your username/password
    authenticate('localhost:7474', 'neo4j', 'password')
    graph.delete_all()

## Elements and Connections

Modeling is all about elements and connections. Similarly, graphs are all about nodes and relationships.

However, we'll draw a clear distinction between element/connection and node/relationship right now - **both elements and connections are nodes.**

### Connections are Nodes?

In Neo4j, relationships do not have relationships. Relationships know little more than a string label.

On the other hand, connections have a lot of information in them - a connection's source and target are specific elements, and its generalization and specializations are other specific connections.

Because a connection has relationships to those elements and connections, it cannot be a relationship itself.

## Metamodel Elements

We need a metamodel - the logical framework that will eventually structure our model.

For this project, we'll use just two elements and one connection from UML, but the entirety of SysML can be defined in this same way.

Let's define Class and Interface.

In [2]:
from py2neo import Node
from scripts.vis import draw

# Neo4j nodes are defined by labels and properties
class_ = Node(
    'metamodel',   # label
    'element',     # label
    name='Class',  # property
    abstract=True  # property
)

# This saves the node to the graph database
graph.create(class_)

interface = Node('metamodel', 'element', name='Interface')
graph.create(interface)

# This function draws all of the graph's nodes and relationships in a bouncy vis.js visualization - drag the nodes around!
draw(graph)

## Metamodel Connections

The metamodel also contains connections between its elements, such as "a Class composes a Class", "a Class implements an Interface", and "a Class is associated with a Class".

Let's create these connections.

In [3]:
composes = Node('metamodel', 'connection', name='Composes')
graph.create(composes)

implements = Node('metamodel', 'connection', name='Implements')
graph.create(implements)

is_associated_with = Node('metamodel', 'connection', name='Is Associated With')
graph.create(is_associated_with)

draw(graph)

Now that we have the elements and connections, we define which elements are the sources and targets of which connections.

In [4]:
from py2neo import Relationship

# Neo4j relationships are mostly defined by a string label, as relationships do not inherit or relate to anything else.
IS_THE_SOURCE_OF = 'IS_THE_SOURCE_OF'
IS_THE_TARGET_OF = 'IS_THE_TARGET_OF'

graph.create(Relationship(class_, IS_THE_SOURCE_OF, composes))
graph.create(Relationship(class_, IS_THE_TARGET_OF, composes))

graph.create(Relationship(class_,    IS_THE_SOURCE_OF, implements))
graph.create(Relationship(interface, IS_THE_TARGET_OF, implements))

graph.create(Relationship(class_, IS_THE_SOURCE_OF, is_associated_with))
graph.create(Relationship(class_, IS_THE_TARGET_OF, is_associated_with))

# Unfortunately, if source & target are the same, vis.js draws the "IS_THE_SOURCE_OF" on top of the "IS_THE_TARGET_OF".
draw(graph)

### Voila! A fully-queryable metamodel!

***Queryable?***

So far the only visualization we've seen has been the bouncy vis.js networks - let's ask Neo4j some questions.

Neo4j queries are performed with the [Cypher Query Language](http://neo4j.com/docs/stable/cypher-query-lang.html).

Much of a Cypher query looks like you're drawing a graph between `(nodes) -[relationships]-> (and other nodes)`, with a few `WHERE` and `AS` keywords thrown in.

Both nodes and relationships can have labels and properties, so we add those labels and properties to the "drawings" in our queries, in order to specify which nodes and relationships we're looking for.

In [5]:
# Refer to a node: (variable_name)
graph.cypher.execute("MATCH (nodes) RETURN nodes.name")

   | nodes.name        
---+--------------------
 1 | Class             
 2 | Interface         
 3 | Composes          
 4 | Implements        
 5 | Is Associated With

In [6]:
# Specify a node label: (variable_name:label)
graph.cypher.execute("MATCH (elements:element) RETURN elements.name")

   | elements.name
---+---------------
 1 | Class        
 2 | Interface    

In [7]:
# Specify a node property: (variable_name:label {property:value})
graph.cypher.execute("MATCH (class {name:'Class'}) RETURN class.abstract")

   | class.abstract
---+----------------
 1 |           True

In [8]:
# Specify a relationship type: -[variable_name:type]->
graph.cypher.execute("""
MATCH (sources) -[:IS_THE_SOURCE_OF]-> (connections) <-[:IS_THE_TARGET_OF]- (targets)
RETURN connections.name as Connections, sources.name as Sources, targets.name as Targets
""")

   | Connections        | Sources | Targets  
---+--------------------+---------+-----------
 1 | Is Associated With | Class   | Class    
 2 | Composes           | Class   | Class    
 3 | Implements         | Class   | Interface

### What about generalization and specialization?

*Good question! I'm still struggling with this a bit, but here's my current thinking:*

Specialization is better represented by a relationship than a node.

Let's say element A specializes element B and specialization is represented by a connection node. To connect A to B, we must create a new specialization node "A Specializes B" that specializes the base "Specializes" node. To connect "A Specializes B" to "Specializes", we must create a new specialization node... etc.

Now, whether "specialization as a relationship" reveals a flaw in this graph database's design or a useful embedding of a common metamodel pattern into the fabric of the graph database is for you to decide after you see specialization in action when we define our model! Let's continue.

## Model Elements

Let's define our project model - in order to find Jack a kidney donor, we'll need to model the following:

* humans
* kidneys
* blood types
* friendship
* dogs
* octopuses

*Obviously we need to model dogs and octopuses in order to be thorough about this kidney search.*

Let's figure out what metamodel elements we can specialize in our model.

In [9]:
graph.cypher.execute("MATCH (elements:element) RETURN elements.name as Elements")

   | Elements 
---+-----------
 1 | Class    
 2 | Interface

Lets make an Animal Class and a Vertebrate Interface!

*Note: The only requirement for being a vertebrate is an internal skeleton... conveniently for us, vertebrates are also the only animals with kidneys!*

What properties are already defined for Class and Interface?

In [10]:
graph.cypher.execute("""
MATCH (class {name:'Class'}), (interface {name:'Interface'})
RETURN keys(class) as `Class Properties`, keys(interface) as `Interface Properties`
""")

   | Class Properties     | Interface Properties
---+----------------------+----------------------
 1 | ['name', 'abstract'] | ['name']            

Animals have a blood type, so we'll need to define this property as `None`, to be redefined by its specializations.

*Note: I'm going to drop the "abstract" property from now on, because I'm not quite sure yet how I want that to work.*

In [11]:
SPECIALIZES = 'SPECIALIZES'

animal = Node('model', 'element', name='Animal', blood_type=None)

# Creating a relationship with an un-created node automatically creates that node
graph.create(Relationship(animal, SPECIALIZES, class_))

vertebrate = Node('model', 'element', name='Vertebrate')
graph.create(Relationship(vertebrate, SPECIALIZES, interface))

draw(graph, labels=['model'])

In [12]:
graph.cypher.execute("MATCH (elements:element) RETURN elements.name as Elements")

   | Elements  
---+------------
 1 | Class     
 2 | Interface 
 3 | Animal    
 4 | Vertebrate

In [13]:
human = Node('model', 'element', name='Human', blood_type=None)
graph.create(Relationship(human, SPECIALIZES, animal))

dog = Node('model', 'element', name='Dog', blood_type=None)
graph.create(Relationship(dog, SPECIALIZES, animal))

octopus = Node('model', 'element', name='Octopus', blood_type='Invertebrate Blood')
graph.create(Relationship(octopus, SPECIALIZES, animal))

organ = Node('model', 'element', name='Organ')
graph.create(Relationship(organ, SPECIALIZES, class_))

kidney = Node('model', 'element', name='Kidney')
graph.create(Relationship(kidney, SPECIALIZES, organ))

draw(graph)

## Model Connections

Let's specialize the metamodel connections. What connections can we specialize, and between what elements?

In [14]:
graph.cypher.execute("""
MATCH (potential_sources) -[:SPECIALIZES*]-> () -[:IS_THE_SOURCE_OF]-> (connections)
WHERE potential_sources:model AND connections:metamodel
RETURN connections.name as Connection, potential_sources.name as `Potential Sources`
""")

    | Connection         | Potential Sources
----+--------------------+-------------------
  1 | Composes           | Animal           
  2 | Composes           | Human            
  3 | Composes           | Dog              
  4 | Composes           | Octopus          
  5 | Composes           | Organ            
  6 | Composes           | Kidney           
  7 | Implements         | Animal           
  8 | Implements         | Human            
  9 | Implements         | Dog              
 10 | Implements         | Octopus          
 11 | Implements         | Organ            
 12 | Implements         | Kidney           
 13 | Is Associated With | Animal           
 14 | Is Associated With | Human            
 15 | Is Associated With | Dog              
 16 | Is Associated With | Octopus          
 17 | Is Associated With | Organ            
 18 | Is Associated With | Kidney           

In [15]:
graph.cypher.execute("""
MATCH (potential_targets) -[:SPECIALIZES*]-> () -[:IS_THE_TARGET_OF]-> (connections)
WHERE potential_targets:model AND connections:metamodel
RETURN connections.name as Connection, potential_targets.name as `Potential Targets`
""")

    | Connection         | Potential Targets
----+--------------------+-------------------
  1 | Composes           | Animal           
  2 | Composes           | Human            
  3 | Composes           | Dog              
  4 | Composes           | Octopus          
  5 | Composes           | Organ            
  6 | Composes           | Kidney           
  7 | Implements         | Vertebrate       
  8 | Is Associated With | Animal           
  9 | Is Associated With | Human            
 10 | Is Associated With | Dog              
 11 | Is Associated With | Octopus          
 12 | Is Associated With | Organ            
 13 | Is Associated With | Kidney           

In [16]:
human__implements__vertebrate = Node('model', 'connection', name='Human Implements Vertebrate')
graph.create(Relationship(human__implements__vertebrate, SPECIALIZES, implements))
graph.create(Relationship(human,      IS_THE_SOURCE_OF, human__implements__vertebrate))
graph.create(Relationship(vertebrate, IS_THE_TARGET_OF, human__implements__vertebrate))

dog__implements__vertebrate = Node('model', 'connection', name='Dog Implements Vertebrate')
graph.create(Relationship(dog__implements__vertebrate, SPECIALIZES, implements))
graph.create(Relationship(dog,        IS_THE_SOURCE_OF, dog__implements__vertebrate))
graph.create(Relationship(vertebrate, IS_THE_TARGET_OF, dog__implements__vertebrate))

organ__composes__animal = Node('model', 'connection', name='Organ Is Inside')
graph.create(Relationship(organ__composes__animal, SPECIALIZES, composes))
graph.create(Relationship(organ,  IS_THE_SOURCE_OF, organ__composes__animal))
graph.create(Relationship(animal, IS_THE_TARGET_OF, organ__composes__animal))

kidney_is_inside = Node('model', 'connection', name='Kidney Is Inside')
graph.create(Relationship(kidney_is_inside, SPECIALIZES, composes))
graph.create(Relationship(kidney,     IS_THE_SOURCE_OF, kidney_is_inside))
graph.create(Relationship(vertebrate, IS_THE_TARGET_OF, kidney_is_inside))

is_friends_with = Node('model', 'connection', name='Is Friends With')
graph.create(Relationship(is_friends_with, SPECIALIZES, is_associated_with))
graph.create(Relationship(animal, IS_THE_SOURCE_OF, is_friends_with))
graph.create(Relationship(animal, IS_THE_TARGET_OF, is_friends_with))

draw(graph)

Zoom in/out (by scrolling), pan (by dragging the white background), and drag nodes around to get a better view of all these graphs.

## Instance Elements

It's finally time for us to input data about Jack, his friends, and their kidneys! What elements can we create?

In [17]:
graph.cypher.execute("MATCH (elements:element) WHERE elements:model RETURN elements.name as Elements")

   | Elements  
---+------------
 1 | Animal    
 2 | Vertebrate
 3 | Human     
 4 | Dog       
 5 | Octopus   
 6 | Organ     
 7 | Kidney    

In [18]:
jack = Node('instance', 'element', name='Jack', blood_type='A+')
graph.create(Relationship(jack, SPECIALIZES, human))

jill = Node('instance', 'element', name='Jill', blood_type='B-')
graph.create(Relationship(jill, SPECIALIZES, human))

john = Node('instance', 'element', name='John')
graph.create(Relationship(john, SPECIALIZES, human))

jane = Node('instance', 'element', name='Jane', blood_type='A+')
graph.create(Relationship(jane, SPECIALIZES, human))

jasper = Node('instance', 'element', name='Jasper', blood_type='DEA 1.1')
graph.create(Relationship(jasper, SPECIALIZES, dog))

jacoby = Node('instance', 'element', name='Jacoby')
graph.create(Relationship(jacoby, SPECIALIZES, octopus))

jack_left_kidney = Node('instance', 'element', name="Jack's Left Kidney")
graph.create(Relationship(jack_left_kidney, SPECIALIZES, kidney))

jill_right_kidney = Node('instance', 'element', name="Jill's Right Kidney")
graph.create(Relationship(jill_right_kidney, SPECIALIZES, kidney))

john_left_kidney = Node('instance', 'element', name="John's Left Kidney")
graph.create(Relationship(john_left_kidney, SPECIALIZES, kidney))

john_right_kidney = Node('instance', 'element', name="John's Right Kidney")
graph.create(Relationship(john_right_kidney, SPECIALIZES, kidney))

jane_left_kidney = Node('instance', 'element', name="Jane's Left Kidney")
graph.create(Relationship(jane_left_kidney, SPECIALIZES, kidney))

jane_right_kidney = Node('instance', 'element', name="Jane's Right Kidney")
graph.create(Relationship(jane_right_kidney, SPECIALIZES, kidney))

jasper_left_kidney = Node('instance', 'element', name="Jasper's Left Kidney")
graph.create(Relationship(jasper_left_kidney, SPECIALIZES, kidney))

jasper_right_kidney = Node('instance', 'element', name="Jasper's Right Kidney")
graph.create(Relationship(jasper_right_kidney, SPECIALIZES, kidney))

draw(graph)

## Instance Connections

What model connections can we specialize, and between what elements?

In [19]:
graph.cypher.execute("""
MATCH (potential_sources) -[:SPECIALIZES*]-> () -[:IS_THE_SOURCE_OF]-> (connections)
WHERE potential_sources:instance AND connections:model
RETURN connections.name as Connection, potential_sources.name as `Potential Sources`
""")

    | Connection                  | Potential Sources    
----+-----------------------------+-----------------------
  1 | Human Implements Vertebrate | Jack                 
  2 | Human Implements Vertebrate | Jill                 
  3 | Human Implements Vertebrate | John                 
  4 | Human Implements Vertebrate | Jane                 
  5 | Dog Implements Vertebrate   | Jasper               
  6 | Organ Is Inside             | Jack's Left Kidney   
  7 | Organ Is Inside             | Jill's Right Kidney  
  8 | Organ Is Inside             | John's Left Kidney   
  9 | Organ Is Inside             | John's Right Kidney  
 10 | Organ Is Inside             | Jane's Left Kidney   
 11 | Organ Is Inside             | Jane's Right Kidney  
 12 | Organ Is Inside             | Jasper's Left Kidney 
 13 | Organ Is Inside             | Jasper's Right Kidney
 14 | Kidney Is Inside            | Jack's Left Kidney   
 15 | Kidney Is Inside            | Jill's Right Kidney  
 16 | Kidney 

In [20]:
graph.cypher.execute("""
MATCH (potential_targets) -[:SPECIALIZES*]-> () -[:IS_THE_TARGET_OF]-> (connections)
WHERE potential_targets:instance AND connections:model
RETURN connections.name as Connection, potential_targets.name as `Potential Targets`
""")

    | Connection      | Potential Targets
----+-----------------+-------------------
  1 | Organ Is Inside | Jack             
  2 | Organ Is Inside | Jill             
  3 | Organ Is Inside | John             
  4 | Organ Is Inside | Jane             
  5 | Organ Is Inside | Jasper           
  6 | Organ Is Inside | Jacoby           
  7 | Is Friends With | Jack             
  8 | Is Friends With | Jill             
  9 | Is Friends With | John             
 10 | Is Friends With | Jane             
 11 | Is Friends With | Jasper           
 12 | Is Friends With | Jacoby           

In [21]:
jack_left_kidney__kidney_is_inside__jack = Node('instance', 'connection', name="Jack's Left Kidney Is Inside Jack")
graph.create(Relationship(jack_left_kidney__kidney_is_inside__jack, SPECIALIZES, kidney_is_inside))
graph.create(Relationship(jack_left_kidney, IS_THE_SOURCE_OF, jack_left_kidney__kidney_is_inside__jack))
graph.create(Relationship(jack,             IS_THE_TARGET_OF, jack_left_kidney__kidney_is_inside__jack))

jill_right_kidney__kidney_is_inside__jill = Node('instance', 'connection', name="Jill's Right Kidney Is Inside Jill")
graph.create(Relationship(jill_right_kidney__kidney_is_inside__jill, SPECIALIZES, kidney_is_inside))
graph.create(Relationship(jill_right_kidney, IS_THE_SOURCE_OF, jill_right_kidney__kidney_is_inside__jill))
graph.create(Relationship(jill,              IS_THE_TARGET_OF, jill_right_kidney__kidney_is_inside__jill))

john_left_kidney__kidney_is_inside__john = Node('instance', 'connection', name="John's Left Kidney Is Inside John")
graph.create(Relationship(john_left_kidney__kidney_is_inside__john, SPECIALIZES, kidney_is_inside))
graph.create(Relationship(john_left_kidney, IS_THE_SOURCE_OF, john_left_kidney__kidney_is_inside__john))
graph.create(Relationship(john,             IS_THE_TARGET_OF, john_left_kidney__kidney_is_inside__john))

john_right_kidney__kidney_is_inside__john = Node('instance', 'connection', name="John's Right Kidney Is Inside John")
graph.create(Relationship(john_right_kidney__kidney_is_inside__john, SPECIALIZES, kidney_is_inside))
graph.create(Relationship(john_right_kidney, IS_THE_SOURCE_OF, john_right_kidney__kidney_is_inside__john))
graph.create(Relationship(john,              IS_THE_TARGET_OF, john_right_kidney__kidney_is_inside__john))

jane_left_kidney__kidney_is_inside__jane = Node('instance', 'connection', name="Jane's Left Kidney Is Inside Jane")
graph.create(Relationship(jane_left_kidney__kidney_is_inside__jane, SPECIALIZES, kidney_is_inside))
graph.create(Relationship(jane_left_kidney, IS_THE_SOURCE_OF, jane_left_kidney__kidney_is_inside__jane))
graph.create(Relationship(jane,             IS_THE_TARGET_OF, jane_left_kidney__kidney_is_inside__jane))

jane_right_kidney__kidney_is_inside__jane = Node('instance', 'connection', name="Jane's Right Kidney Is Inside Jane")
graph.create(Relationship(jane_right_kidney__kidney_is_inside__jane, SPECIALIZES, kidney_is_inside))
graph.create(Relationship(jane_right_kidney, IS_THE_SOURCE_OF, jane_right_kidney__kidney_is_inside__jane))
graph.create(Relationship(jane,              IS_THE_TARGET_OF, jane_right_kidney__kidney_is_inside__jane))

jasper_left_kidney__kidney_is_inside__jasper = Node('instance', 'connection', name="Jasper's Left Kidney Is Inside Jasper")
graph.create(Relationship(jasper_left_kidney__kidney_is_inside__jasper, SPECIALIZES, kidney_is_inside))
graph.create(Relationship(jasper_left_kidney, IS_THE_SOURCE_OF, jasper_left_kidney__kidney_is_inside__jasper))
graph.create(Relationship(jasper,             IS_THE_TARGET_OF, jasper_left_kidney__kidney_is_inside__jasper))

jasper_right_kidney__kidney_is_inside__jasper = Node('instance', 'connection', name="Jasper's Left Kidney Is Inside Jasper")
graph.create(Relationship(jasper_right_kidney__kidney_is_inside__jasper, SPECIALIZES, kidney_is_inside))
graph.create(Relationship(jasper_right_kidney, IS_THE_SOURCE_OF, jasper_right_kidney__kidney_is_inside__jasper))
graph.create(Relationship(jasper,              IS_THE_TARGET_OF, jasper_right_kidney__kidney_is_inside__jasper))

jack__is_friends_with__jill = Node('instance', 'connection', name='Jack Is Friends With Jill')
graph.create(Relationship(jack__is_friends_with__jill, SPECIALIZES, is_friends_with))
graph.create(Relationship(jack, IS_THE_SOURCE_OF, jack__is_friends_with__jill))
graph.create(Relationship(jill, IS_THE_TARGET_OF, jack__is_friends_with__jill))

jill__is_friends_with__john = Node('instance', 'connection', name='Jill Is Friends With John')
graph.create(Relationship(jill__is_friends_with__john, SPECIALIZES, is_friends_with))
graph.create(Relationship(jill, IS_THE_SOURCE_OF, jill__is_friends_with__john))
graph.create(Relationship(john, IS_THE_TARGET_OF, jill__is_friends_with__john))

john__is_friends_with__jane = Node('instance', 'connection', name='John Is Friends With Jane')
graph.create(Relationship(john__is_friends_with__jane, SPECIALIZES, is_friends_with))
graph.create(Relationship(john, IS_THE_SOURCE_OF, john__is_friends_with__jane))
graph.create(Relationship(jane, IS_THE_TARGET_OF, john__is_friends_with__jane))

jack__is_friends_with__jasper = Node('instance', 'connection', name='Jack Is Friends With Jasper')
graph.create(Relationship(jack__is_friends_with__jasper, SPECIALIZES, is_friends_with))
graph.create(Relationship(jack,   IS_THE_SOURCE_OF, jack__is_friends_with__jasper))
graph.create(Relationship(jasper, IS_THE_TARGET_OF, jack__is_friends_with__jasper))

jack__is_friends_with__jacoby = Node('instance', 'connection', name='Jack Is Friends With Jacoby')
graph.create(Relationship(jack__is_friends_with__jacoby, SPECIALIZES, is_friends_with))
graph.create(Relationship(jack,   IS_THE_SOURCE_OF, jack__is_friends_with__jacoby))
graph.create(Relationship(jacoby, IS_THE_TARGET_OF, jack__is_friends_with__jacoby))


draw(graph, relax_gray_relationships=True)

### Voilalala! A fully-queryable metamodel/model/instance database

Let's ask our database some questions to learn about Jack's situation:

* What is Jack's blood type?
* Who has Jack's blood type?
* Who has a spare kidney?
* Who doesn't have any kidneys?
* Who is Jack friends with?
* Who can Jack become friends with?
* Whose blood type do we not know?

In [22]:
graph.cypher.execute("MATCH (jack {name:'Jack'}) RETURN jack.blood_type as `Jack's blood type`")

   | Jack's blood type
---+-------------------
 1 | A+               

In [23]:
graph.cypher.execute("""
MATCH
(jack {name:'Jack'}), (others_with_jack_blood_type)
WHERE
others_with_jack_blood_type.blood_type = jack.blood_type AND
others_with_jack_blood_type.name <> jack.name
RETURN
others_with_jack_blood_type.name AS `Others with Jack's blood type`
""")

   | Others with Jack's blood type
---+-------------------------------
 1 | Jane                         

In [24]:
graph.cypher.execute("""
MATCH
(others_with_a_kidney) -[:IS_THE_TARGET_OF]-> () -[:SPECIALIZES]-> ({name:'Kidney Is Inside'})
WHERE
others_with_a_kidney.name <> 'Jack'
RETURN
others_with_a_kidney.name AS `Others with a kidney`,
count(*) AS `Number of kidneys` ORDER BY `Number of kidneys` DESC
""")

   | Others with a kidney | Number of kidneys
---+----------------------+-------------------
 1 | Jane                 |                 2
 2 | Jasper               |                 2
 3 | John                 |                 2
 4 | Jill                 |                 1

In [25]:
# Who's the cutest invertebrate? You are!
graph.cypher.execute("""
MATCH
(animals_without_kidneys) -[:SPECIALIZES]-> (animal_specialization) -[:SPECIALIZES]-> ({name:'Animal'})
WHERE
NOT (animal_specialization) -[:`IS_THE_SOURCE_OF`|`IS_THE_TARGET_OF`*3]- ({name:'Kidney Is Inside'})
RETURN
animals_without_kidneys.name AS `Animals without kidneys`
""")

   | Animals without kidneys
---+-------------------------
 1 | Jacoby                 

In [26]:
graph.cypher.execute("""
MATCH
({name:'Jack'}) -[:IS_THE_SOURCE_OF]-> (friendships) -[:SPECIALIZES]-> ({name:'Is Friends With'}),
(friends)       -[:IS_THE_TARGET_OF]-> (friendships)
RETURN
friends.name as `Jack's immediate friends`
""")

   | Jack's immediate friends
---+--------------------------
 1 | Jacoby                  
 2 | Jasper                  
 3 | Jill                    

In [27]:
graph.cypher.execute("""
MATCH
({name:'Jack'}) -[:IS_THE_SOURCE_OF]-> (jack_friendships)         <-[:IS_THE_TARGET_OF]- (jack_friends),
(jack_friends)  -[:IS_THE_SOURCE_OF]-> (jack_friends_friendships) <-[:IS_THE_TARGET_OF]- (jack_friends_friends),
(jack_friendships) -[:SPECIALIZES]->   ({name:'Is Friends With'}) <-[:SPECIALIZES]- (jack_friends_friendships)
RETURN
DISTINCT jack_friends_friends.name as `Friends of Jack's friends`
""")

   | Friends of Jack's friends
---+---------------------------
 1 | John                     

In [28]:
graph.cypher.execute("""
MATCH
(humans) -[:SPECIALIZES]-> ({name:"Human"})
WHERE
humans.blood_type IS NULL
RETURN
humans.name AS `Humans with an unknown blood type`
""")

   | Humans with an unknown blood type
---+-----------------------------------
 1 | John                             

##### So what?

We have identified a recommendation ("Jane is a viable kidney donor") and an unknown ("What is John's blood type?").

##### Why this approach?

This graph database implementation contains all of the information required to support a modeling user interface.

*From this point on, I'll use the word "user" to mean modelers, subject matter experts, and project managers, but not software developers.*

User views of the project model/data should never look like the crowded visualizations above. Users should only interact with diagrams that focus the visualizations and constrain their actions to legal ones.

User actions (creating/editing/deleting an element or connection) should be limited by:

* the diagram's subset of nodes
* the diagram's subset of relationships
* the existing project data in the database
* the model's definitions of what elements/connections can be made
* the metamodel's definitions of what elements/connections can be made

The metamodel/model/instance graph implementation described on this page provides a technologically-minimalist modeling tool with the logical complexity to express any project's model/data and constrain any diagramming interface's user actions.

I look forward to implementing such a diagramming interface and asking the database questions like:

* What are all undefined properties for elements in this diagram?
* For the currently-selected element, what are all elements to which it can connect?
* What are all the possible "Connect X to Y with Z" buttons for the toolbar of this diagram?

**Thanks for reading! I'd love to hear your thoughts/questions about anything on this page.**

\- Jan